# Anthropogenic heat from buildings

This notebook is used to process building energy use, analyze data, and ultimately produce the anthropogenic heat flux (AHF) from buildings for the Greater Los Angeles (LA) region.

In [ ]:
# load packages 
%matplotlib inline
import os
from pathlib import Path
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
#import requests
import geopandas as gpd
import pyproj
import shapely.geometry
import contextily as ctx
from matplotlib.ticker import FormatStrFormatter
import geofeather
import dask_geopandas as dgpd
import pyogrio

## TESTS

In [ ]:
%%time
# regular geopandas
parcels_usc_dir = '/Users/josephko/USC/Research/Anthropogenic Heat/boundaries/LA_County_Parcels_usc.geojson'
parcels_usc_df = gpd.read_file(parcels_usc_dir)

In [ ]:
%%time
# dask geopandas
parcels_usc_dir = '/Users/josephko/USC/Research/Anthropogenic Heat/boundaries/LA_County_Parcels.gpkg'
parcels_usc_df = dgpd.read_file(parcels_usc_dir, npartitions = 1000)

In [ ]:
parcels_usc_df.columns

In [ ]:
%%time
#100 partitions
partition_0 = parcels_usc_df.partitions[0].compute()

In [ ]:
%%time
#1000 partitions
partition_0 = parcels_usc_df.partitions[0].compute()

In [ ]:
partition_0.plot()

In [ ]:
parcels_neighborhoods = parcels_usc_df.sjoin(neighborhoods)


In [ ]:
parcels_indexed_by_hoods = parcels_neighborhoods.set_index('name')

In [ ]:
list(test.columns)

In [ ]:
neighborhoods.columns

In [ ]:
# test clip with small polygons (University Park + Adams-Normandie)
df_clipped = parcels_usc_df.clip(neighborhoods_2)

In [ ]:
df_clipped = df_clipped.compute()

In [ ]:
ddf_filtered.memory_usage(deep=True).sum().compute()

In [ ]:
ddf_filtered = parcels_usc_df.loc[parcels_usc_df["OBJECTID"] < 10]
df_filtered = ddf_filtered.compute()
df_filtered

## Import data from data repositories

Download Energy Atlas data from Kaggle repo (if not already in local directory)

In [ ]:
energy_atlas_data_path = Path('data/energy_atlas')
if not os.path.isdir(energy_atlas_data_path): # if data directory does not exist (i.e., data not downloaded yet)
    # initialize kaggle API
    from kaggle.api.kaggle_api_extended import KaggleApi
    api = KaggleApi()
    api.authenticate()

    # download from my Kaggle dataset page
    dataset = 'josephko/la-energy-atlas-2016'
    download_path = Path('data/energy_atlas')
    api.dataset_download_files(dataset, download_path) # downloads all data in zip file
    
    # unzip and remove zip file
    zip_file = download_path / 'la-energy-atlas-2016.zip'
    with zipfile.ZipFile(zip_file) as file:
        file.extractall(download_path)
    os.remove(zip_file)


Import Energy Atlas data into dataframes

In [ ]:
usage_file = energy_atlas_data_path / 'usage_bld_btu.csv'
usage_bld_btu = pd.read_csv(usage_file, na_values = ['masked'])
#usage_bld_btu = usage_bld_btu.loc[usage_bld_btu['usage'] != 'masked'].copy()

# # TESTING 
# cols = usage_bld_btu.columns.drop(['geo_id', 'usetype', 'name'])
# usage_bld_btu.loc[:,cols] = usage_bld_btu.loc[:, cols].apply(pd.to_numeric, errors = 'coerce')

usage_bld_btu

In [ ]:
usage_bld_btu.dtypes

Import geos.csv which has auxiliary information e.g., city names

In [ ]:
geos_file = energy_atlas_data_path / 'geos.csv'
geos = pd.read_csv(geos_file, na_values = ['NaN'])
geos

Merge consumption data with geos information, and also add additional useful columns

In [ ]:
usage_bld = pd.merge(usage_bld_btu, geos, how='left', on = 'geo_id')
usage_bld.columns

## Import building footpring (LARIAC) and parcels data

IMPORTANT NOTE:<br>
LARIAC height and area (i.e., building footprint) area in units of FEET not meters

Loading the whole LARIAC and parcels datasets takes a lot of memory. 
For now, load sample LARIAC and parcels for the University Park (USC) subset. 

In [ ]:
"""
Note: the shape_area and shape_length attributes are INCORRECT when downloaded from https://geohub.lacity.org/datasets/lacounty::lariac5-buildings-2017
Need to delete those columns are re-calculate shape area based on geometry
"""
lariac_usc_dir = '/Users/josephko/USC/Research/Anthropogenic Heat/lariac/Countywide_Building_Outlines_2017_usc.geojson'
lariac_usc_df = gpd.read_file(lariac_dir)

In [ ]:
#lariac_usc_df.explore()

In [ ]:
"""
How to use ArcGIS REST API to query data
2000 data features limit.
Need to loop to get all data.
"""
# url = 'https://services.arcgis.com/RmCCgQtiZLDCtblq/arcgis/rest/services/Countywide_Building_Outlines_(2017)/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json&resultOffset=3262000'
# test = gpd.read_file(url)
# test.head()

In [ ]:
parcels_usc_dir = '/Users/josephko/USC/Research/Anthropogenic Heat/boundaries/LA_County_Parcels_usc.geojson'
parcels_usc_df = gpd.read_file(parcels_usc_dir, na_values = ['masked'])
parcels_usc_df.plot()

Filter columns to reduce clutter

In [ ]:
#lariac_usc_df.head()

In [ ]:
# rename OBJECTID to avoid duplicate column names
lariac_usc_df.rename(columns={'OBJECTID':'lariac_id'}, inplace=True)
parcels_usc_df.rename(columns={'OBJECTID':'parcel_id'}, inplace=True)

In [ ]:
# choose columns to keep
lariac_columns = ['lariac_id',
 'CODE',
 'BLD_ID',
 'HEIGHT',
 'ELEV',
 'DATE_',
 'STATUS',
 'AREA',
 'geometry']

parcels_columns = ['parcel_id',
 'AIN',
 'APN',
 'UseType',
 'UseDescrip',
 'DesignType',
 'CENTER_LAT',
 'CENTER_LON',
 'CENTER_X',
 'CENTER_Y',
 'LAT_LON',
 'ShapeSTAre',
 'ShapeSTLen',
 'geometry']

In [ ]:
# keep only columns listed above
lariac_usc_df = lariac_usc_df[lariac_columns]
parcels_usc_df = parcels_usc_df[parcels_columns]

In [ ]:
# calculate shape area from geometry in units of meters squared
lariac_usc_df['shape_area_whole'] = lariac_usc_df.to_crs('epsg:3857').geometry.area

In [ ]:
# delete courtyards, keep only buildings
lariac_usc_df.drop(lariac_usc_df[lariac_usc_df['CODE']=='Courtyard'].index, inplace=True)

In [ ]:
#lariac_usc_df.shape

In [ ]:
#len(lariac_usc_df['lariac_id'].unique())

In [ ]:
#lariac_usc_df.explore()

## Download LA Times Mapping neighborhoods data

In [ ]:
# source: https://usc.data.socrata.com/dataset/Los-Angeles-Neighborhood-Map/r8qd-yxsr
neighborhoods_dir = '/Users/josephko/USC/Research/Anthropogenic Heat/boundaries/la_bound_nbhd.geojson'
neighborhoods = gpd.read_file(neighborhoods_dir)
neighborhoods.plot()

In [ ]:
neighborhoods.dtypes

In [ ]:
# county_boundary_url = 'https://opendata.arcgis.com/datasets/10f1e37c065347e693cf4e8ee753c09b_15.geojson'
# county_boundary = gpd.read_file(county_boundary_url)
# county_boundary.plot()

In [ ]:
# This file contains neighborhoods in Orange County as well. Used file saved on laptop downloaded from website (See above).
# url = 'https://github.com/datadesk/mapping-la-data/raw/main/geojson/la-county-neighborhoods-v6.geojson'
# #file = requests.get(url)
# neighborhoods = gpd.read_file(url)
# neighborhoods.head()

In [ ]:
# plot boundaries of neighborhoods for visual check
neighborhoods.plot()

In [ ]:
neighborhoods.crs

In [ ]:
# Remove Catalina Island (including Avalon) from dataframe
catalina_index = neighborhoods.query('name.str.contains("Catalina") | name.str.contains("Avalon")').index
neighborhoods.drop(catalina_index, inplace=True)

In [ ]:
# Check that it was removed
print(neighborhoods.query('name.str.contains("Catalina") | name.str.contains("Avalon")'))
neighborhoods.plot()

In [ ]:
#neighborhoods.explore()

## Merge neighborhood boundaries and Energy Atlas data

In [ ]:
neighborhoods.head()

In [ ]:
usage_bld_btu

In [ ]:
neighborhoods_lower = neighborhoods.copy()
neighborhoods_lower['name'] = neighborhoods_lower['name'].str.lower()
neighborhoods_lower.head()

In [ ]:
usage_bld = neighborhoods_lower.merge(usage_bld_btu, on='name')
usage_bld.columns

In [ ]:
# clean up dataframe, drop unnecessary columns
cols_drop = ['external_i','slug_1','display_na', 'set', 'name_1']
usage_bld.drop(cols_drop, axis=1, inplace=True)
usage_bld.columns

In [ ]:
# add column for usage in units of Watts (1 btu = 0.293 watts)
usage_bld['usage_joules'] = usage_bld['usage']*1055.06

In [ ]:
usage_bld.plot(column='usage_joules', legend=True, figsize=(20,10))

## Descriptive analysis of Energy Atlas data

In [ ]:
# How many different usetype categories?
usage_bld_btu['usetype'].unique()

In [ ]:
# subset only neighborhood-aggregated records
usage_bld_btu_neighborhoods = usage_bld_btu.loc[usage_bld_btu['geo_id'].str.contains('neighborhoods')].copy()
usage_bld_btu_neighborhoods

In [ ]:
# How many unique neighborhoods?
len(usage_bld_btu_neighborhoods['geo_id'].unique())

In [ ]:
# further subset for only residential
usage_bld_btu_neighborhoods_res = usage_bld_btu_neighborhoods.loc[usage_bld_btu_neighborhoods['usetype'] == 'res_total'].copy()
usage_bld_btu_neighborhoods_res

In [ ]:
print(usage_bld_btu_neighborhoods_res._is_copy)

In [ ]:
# testing SettingWithCopyWarning with test df
df = pd.DataFrame({'A': ['1', '2', 'NaN'], 'B': ['1', 'NaN', '3'], 'C': ['1', '2', '3']})
cols = df.columns.drop(['C'])
df.loc[:, cols] = df[cols].apply(pd.to_numeric, errors = 'coerce')
df

In [ ]:
# convert appropriate columns to numeric
cols = usage_bld_btu_neighborhoods_res.columns.drop(['geo_id', 'usetype', 'name'])
usage_bld_btu_neighborhoods_res.loc[:,cols] = usage_bld_btu_neighborhoods_res[cols].apply(pd.to_numeric, errors = 'coerce')
usage_bld_btu_neighborhoods_res.dtypes

In [ ]:
# distribution of pct_elec
# usage_bld_btu_neighborhoods_res['pct_elec'] = pd.to_numeric(usage_bld_btu_neighborhoods_res['pct_elec'])
usage_bld_btu_neighborhoods_res['pct_elec'].plot.hist(bins = 20) 

In [ ]:
# distribution of pop
usage_bld_btu_neighborhoods_res['pop'].plot.hist(bins = 20) 

In [ ]:
# distribution of usage
usage_bld_btu_neighborhoods_res['usage'].plot.hist(bins = 20) 

In [ ]:
# distribution of usage per capita
usage_bld_btu_neighborhoods_res['usage_percap'].plot.hist(bins = 20) 

## Create 2d grid

In [ ]:
"""
example references:
https://james-brennan.github.io/posts/fast_gridding_geopandas/
https://stackoverflow.com/questions/40342355/how-can-i-generate-a-regular-geographic-grid-using-python
"""

# create grid that covers the extent of LA County, set at 'X' m resolution
xmin, ymin, xmax, ymax= neighborhoods.total_bounds

# Set up transformers, EPSG:3857 is metric, same as EPSG:900913
to_proxy_transformer = pyproj.Transformer.from_crs('epsg:4326', 'epsg:3857', always_xy=True)
to_original_transformer = pyproj.Transformer.from_crs('epsg:4326', 'epsg:3857', always_xy=True)

# Create corners of rectangle to be transformed to a grid
sw = shapely.geometry.Point((xmin, ymin))
ne = shapely.geometry.Point((xmax, ymax))

stepsize = 1000 # grid resolution in meters

# Project corners to target projection
transformed_sw = to_proxy_transformer.transform(sw.x, sw.y) # Transform NW point to 3857
transformed_ne = to_proxy_transformer.transform(ne.x, ne.y) # .. same for SE



In [ ]:
print(xmin, ymin, xmax, ymax)

In [ ]:
# Iterate over 2D area
grid_cells = []
x = transformed_sw[0]
while x < transformed_ne[0]:
    y = transformed_sw[1]
    while y < transformed_ne[1]:
        x1 = x + stepsize
        y1 = y + stepsize
        cell = shapely.geometry.box(x, y, x1, y1)
        grid_cells.append(cell)
        y += stepsize
    x += stepsize


ah_grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs='epsg:3857')
ah_grid = ah_grid.to_crs('epsg:4326')

ax = neighborhoods.plot(figsize=(24, 16))
plt.autoscale(False)
#ah_grid.plot(ax=ax, facecolor="none", edgecolor='grey')

## Anthropogenic heat flux from residential buildings

### Test case 1: University Park neighborhood

In [ ]:
# get usage for university park neighborhood only
usage_usc = usage_bld[(usage_bld['name'] == 'university park') & (usage_bld['usetype'] == 'res_total')]
#usage_usc = usage_bld[usage_bld['usetype'] == 'res_total']
ax = usage_usc.plot(figsize=(10,10), column = 'usage_joules', alpha=0.1, edgecolor='k')
ax.get_xaxis().get_major_formatter().set_useOffset(False)
#ax.annotate(usage_usc.iloc[0].usage_joules, xy=[pd.to_numeric(top_row['latitude']), pd.to_numeric(top_row['longitude'])])
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=usage_bld.crs) # notice the crs must match between basemap and data

In [ ]:
# create 2d grid
"""
example references:
https://james-brennan.github.io/posts/fast_gridding_geopandas/
https://stackoverflow.com/questions/40342355/how-can-i-generate-a-regular-geographic-grid-using-python
"""

# create grid that covers the extent of LA County, set at 'X' m resolution
xmin, ymin, xmax, ymax = usage_usc.total_bounds

# create a little buffer for extent
xmin = xmin - 0.005
ymin = ymin - 0.005
xmax = xmax + 0.005
ymax = ymax + 0.005

# Set up transformers, EPSG:3857 is metric, same as EPSG:900913
to_proxy_transformer = pyproj.Transformer.from_crs('epsg:4326', 'epsg:3857', always_xy=True)
to_original_transformer = pyproj.Transformer.from_crs('epsg:4326', 'epsg:3857', always_xy=True)

# Create corners of rectangle to be transformed to a grid
sw = shapely.geometry.Point((xmin, ymin))
ne = shapely.geometry.Point((xmax, ymax))

stepsize = 100 # grid resolution in meters

# Project corners to target projection
transformed_sw = to_proxy_transformer.transform(sw.x, sw.y) # Transform NW point to 3857
transformed_ne = to_proxy_transformer.transform(ne.x, ne.y) # .. same for SE

# Iterate over 2D area
grid_cells = []
x = transformed_sw[0]
while x < transformed_ne[0]:
    y = transformed_sw[1]
    while y < transformed_ne[1]:
        x1 = x + stepsize
        y1 = y + stepsize
        cell = shapely.geometry.box(x, y, x1, y1)
        grid_cells.append(cell)
        y += stepsize
    x += stepsize


ah_grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs='epsg:3857')
ah_grid = ah_grid.to_crs('epsg:4326')

ax = usage_usc.plot(figsize=(10,10), alpha=0.1, edgecolor='k')
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=usage_bld.crs) # notice the crs must match between basemap and data
ah_grid.plot(ax=ax, facecolor="none", edgecolor='grey')

In [ ]:
# spatial join LARIAC and parcels dataframe
parcels_usc_res_df = parcels_usc_df[parcels_usc_df['UseType'] == 'Residential'].copy() # filter for only residential
parcels_usc_res_df.drop_duplicates('geometry', inplace=True) # !!!REMOVE DUPLICATE PARCELS!!!
lariac_usc_merged_inner = gpd.sjoin(lariac_usc_df, parcels_usc_res_df, how='inner')

# plot to visualize
ax = lariac_usc_merged_inner.plot(figsize=(10,10), alpha=0.5, facecolor = 'none', edgecolor='blue')
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
ah_grid.plot(ax=ax, facecolor="none", edgecolor='gray')
usage_usc.plot(ax=ax, alpha=0.1, edgecolor='k')

In [ ]:
# spatial join the 2d grid and the neighborhood boundary
grid_usc = gpd.sjoin(ah_grid, usage_usc)
# plot to visualize
ax = lariac_usc_merged_inner.plot(figsize=(10,10), alpha=0.5, facecolor = 'none', edgecolor='blue')
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
grid_usc.plot(ax=ax, facecolor="none", edgecolor='gray')
usage_usc.plot(ax=ax, alpha=0.1, edgecolor='k')

In [ ]:
# drop columns with 'index_*'
grid_usc.drop(list(grid_usc.filter(regex = 'index_')), axis=1, inplace=True)
lariac_usc_merged_inner.drop(list(lariac_usc_merged_inner.filter(regex = 'index_')), axis=1, inplace=True)
# add extra geometry column to save grid geometry for spatial joins
grid_usc['grid_geometry'] = grid_usc.geometry
# make the index a column
grid_usc.reset_index(inplace=True)
# rename index to grid_cell_index
grid_usc.rename(columns={'index':'grid_cell_index'}, inplace=True)

In [ ]:
grid_usc.head()

In [ ]:
# spatial join buildings and grid (intersects)
buildings_usc_intersects = gpd.sjoin(lariac_usc_merged_inner, grid_usc, predicate='intersects')
# plot to visualize
ax = buildings_usc_intersects.plot(figsize=(10,10), alpha=0.5, facecolor = 'none', edgecolor='blue')
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
grid_usc.plot(ax=ax, facecolor="none", edgecolor='gray')
usage_usc.plot(ax=ax, alpha=0.1, edgecolor='k')              

In [ ]:
# # spatial join buildings and grid (within)
# buildings_usc_within = gpd.sjoin(lariac_usc_merged_inner, grid_usc, predicate='within')
# # plot to visualize
# ax = grid_usc.plot(figsize=(10,10), facecolor="none", edgecolor='blue')
# ax.get_xaxis().get_major_formatter().set_useOffset(False)
# ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
# buildings_usc_within.plot(ax=ax, alpha=0.5, facecolor = 'yellow', edgecolor='red')
# usage_usc.plot(ax=ax, alpha=0.1, edgecolor='k')       

In [ ]:
# # spatial join buildings and grid (Switch left/right order this time)
# grid_usc_intersects = gpd.sjoin(grid_usc, lariac_usc_merged_inner, predicate='intersects')
# # plot to visualize
# ax = grid_usc_intersects.plot(figsize=(10,10), alpha=0.5, facecolor = 'yellow', edgecolor='red')
# ax.get_xaxis().get_major_formatter().set_useOffset(False)
# ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
# grid_usc.plot(ax=ax, facecolor="none", edgecolor='blue')
# usage_usc.plot(ax=ax, alpha=0.1, edgecolor='k')  

In [ ]:
buildings_usc_intersects.grid_geometry.plot(facecolor="none", edgecolor='blue')

In [ ]:
len(buildings_usc_intersects.grid_geometry.unique())

In [ ]:
#testing overlay 
buildings_usc_overlay_intersection = lariac_usc_merged_inner.overlay(grid_usc, how='intersection')
# plot to visualize
#buildings_usc_overlay_intersection.explore()

In [ ]:
# total building area in this neighborhood?
total_building_area_usc = buildings_usc_overlay_intersection['AREA'].sum()
total_building_area_usc

In [ ]:
# calculate building volume and add as a new column
buildings_usc_overlay_intersection['VOLUME'] = buildings_usc_overlay_intersection['AREA']*buildings_usc_overlay_intersection['HEIGHT'] # units: cubic feet

In [ ]:
#buildings_usc_overlay_intersection.shape

In [ ]:
#len(buildings_usc_overlay_intersection['lariac_id'].unique())

In [ ]:
# add column that describes the fraction of the whole building that segment represents
buildings_usc_overlay_intersection['building_fraction'] = buildings_usc_overlay_intersection.to_crs('epsg:3857').geometry.area/buildings_usc_overlay_intersection['shape_area_whole']
buildings_usc_overlay_intersection['VOLUME'] = buildings_usc_overlay_intersection['building_fraction']*buildings_usc_overlay_intersection['VOLUME']
#buildings_usc_overlay_intersection

In [ ]:
# total building area in this neighborhood?
total_building_vol_usc = buildings_usc_overlay_intersection['VOLUME'].sum()
total_building_vol_usc

In [ ]:
# df of total building volume in each grid cell
building_volume_df = buildings_usc_overlay_intersection[['grid_cell_index', 'VOLUME']]
volume_by_cell = building_volume_df.groupby('grid_cell_index').sum()
# make the index a column
volume_by_cell.reset_index(inplace=True)

In [ ]:
# add cells with no buildings 
volume_by_cell_all = volume_by_cell.merge(grid_usc, how='outer') # outer join
volume_by_cell_all = volume_by_cell_all[['grid_cell_index','VOLUME','pct_elec','usage_joules','grid_geometry']] # only keep these columns
volume_by_cell_all.head()

In [ ]:
# calculate volume-based weights and finally, the AHF
cell_area = stepsize**2 # area of single cell; stepsize is resolution in meters (defined above)
n_cells = len(grid_usc) # number of total cells in neighborhood
seconds_in_year = 3.154e+7
ahf = gpd.GeoDataFrame(volume_by_cell_all, geometry='grid_geometry')
ahf['VOLUME'] = ahf['VOLUME'].fillna(0)
ahf['vol_frac'] = ahf['VOLUME']/total_building_vol_usc
ahf['electricity_use'] = ahf['pct_elec']*ahf['usage_joules']*ahf['vol_frac']
ahf['ahf'] = ahf['electricity_use']/cell_area/seconds_in_year
ahf

In [ ]:
# plot to visualize
ax = ahf.plot(figsize=(10,10), column='ahf', alpha=0.7, legend=True, cmap='YlOrRd', vmin=0, vmax=6, legend_kwds={'label':'Anthropogenic heat flux [W/$m^2$]'})
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
grid_usc.plot(ax=ax, facecolor="none", edgecolor='gray')
buildings_usc_intersects.plot(ax=ax, facecolor="none", edgecolor='blue', alpha=0.2)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

In [ ]:
# plot AHF with naive averaging (i.e., no building volume weights)
ahf['electricity_use_naive'] = (ahf['pct_elec']*ahf['usage_joules'])/n_cells
ahf['ahf_naive'] = ahf['electricity_use_naive']/cell_area/seconds_in_year

In [ ]:
# plot to visualize
ax = ahf.plot(figsize=(10,10), column='ahf_naive', alpha=0.7, legend=True, cmap='YlOrRd', vmin=0, vmax=6,legend_kwds={'label':'Anthropogenic heat flux [W/$m^2$]'})
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
grid_usc.plot(ax=ax, facecolor="none", edgecolor='gray')
buildings_usc_intersects.plot(ax=ax, facecolor="none", edgecolor='blue', alpha=0.2)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

### Test case 2: University Park and Adams-Normandie

In [ ]:
# create mask for two neighborhoods
neighborhoods_2 = neighborhoods[(neighborhoods['name']=='University Park') | (neighborhoods['name']=='Adams-Normandie')]
neighborhoods_2

In [ ]:
%%time
# load subset of parcels using mask                              
parcels_2_dir = '/Users/josephko/USC/Research/Anthropogenic Heat/boundaries/LA_County_Parcels-shp.zip'
parcels_2_df = gpd.read_file(parcels_2_dir, na_values = ['masked'], mask = neighborhoods_2)
parcels_2_df.plot()

In [ ]:
%%time
# loading all lariac data as geojson                       
lariac_2_dir = '/Users/josephko/USC/Research/Anthropogenic Heat/lariac/Countywide_Building_Outlines_2017.geojson'
lariac_2_df = gpd.read_file(lariac_2_dir)#, mask = neighborhoods_2)

In [ ]:
%%time
# loading all lariac data as gdb                       
lariac_2_dir = '/Users/josephko/USC/Research/Anthropogenic Heat/lariac/Countywide_Building_Outlines_2017.gdb'
lariac_2_df = gpd.read_file(lariac_2_dir)#, mask = neighborhoods_2)

In [ ]:
# convert to feather 
geofeather.to_geofeather(lariac_2_df, '/Users/josephko/USC/Research/Anthropogenic Heat/lariac/Countywide_Building_Outlines_2017.feather')

In [ ]:
%%time
# read feather file
lariac_all_df = geofeather.from_geofeather('/Users/josephko/USC/Research/Anthropogenic Heat/lariac/Countywide_Building_Outlines_2017.feather')

In [ ]:
%%time
# load all parcels                            
parcels_2_dir = '/Users/josephko/USC/Research/Anthropogenic Heat/boundaries/LA_County_Parcels.geojson'
parcels_2_df = gpd.read_file(parcels_2_dir)

In [ ]:
%%time
# convert parcels to feather 
geofeather.to_geofeather(lariac_2_df, '/Users/josephko/USC/Research/Anthropogenic Heat/lariac/Countywide_Building_Outlines_2017.feather')

In [ ]:
%%time
# read parcels feather file
lariac_all_df = geofeather.from_geofeather('/Users/josephko/USC/Research/Anthropogenic Heat/lariac/Countywide_Building_Outlines_2017.feather')

In [ ]:
# rename OBJECTID to avoid duplicate column names
lariac_2_df.rename(columns={'OBJECTID':'lariac_id'}, inplace=True)
parcels_2_df.rename(columns={'OBJECTID':'parcel_id'}, inplace=True)

# keep only columns listed above
lariac_2_df = lariac_2_df[lariac_columns].copy()
parcels_2_df = parcels_2_df[parcels_columns].copy()

# calculate shape area from geometry in units of meters squared
lariac_2_df['shape_area_whole'] = lariac_2_df.to_crs('epsg:3857').geometry.area

# delete courtyards, keep only buildings
lariac_2_df.drop(lariac_2_df[lariac_2_df['CODE']=='Courtyard'].index, inplace=True)

In [ ]:
# get usage for university park neighborhood only
usage_usc = usage_bld[((usage_bld['name'] == 'adams-normandie') |
                      (usage_bld['name'] == 'university park')) &
                      (usage_bld['usetype'] == 'res_total')]
#usage_usc = usage_bld[usage_bld['usetype'] == 'res_total']
ax = usage_usc.plot(figsize=(10,10), column = 'usage_joules', alpha=0.1, edgecolor='k')
ax.get_xaxis().get_major_formatter().set_useOffset(False)
#ax.annotate(usage_usc.iloc[0].usage_joules, xy=[pd.to_numeric(top_row['latitude']), pd.to_numeric(top_row['longitude'])])
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=usage_bld.crs) # notice the crs must match between basemap and data

## Anthropogenic heat flux from commercial buildings

### Proof of concept: University Park neighborhood

In [ ]:
usage_bld['usetype'].unique()

In [ ]:
# get usage for university park neighborhood only
usage_usc_com = usage_bld[(usage_bld['name'] == 'university park') & (usage_bld['usetype'] == 'commercial')]

In [ ]:
usage_usc_com.head()

In [ ]:
# spatial join LARIAC and parcels dataframe
parcels_usc_com_df = parcels_usc_df[parcels_usc_df['UseType'] == 'Commercial'].copy() # filter for only commercial
parcels_usc_com_df.drop_duplicates('geometry', inplace=True) # !!!REMOVE DUPLICATE PARCELS!!!
#lariac_usc_merged_inner_com = gpd.sjoin(lariac_usc_df, parcels_usc_com_df, how='inner')
lariac_usc_merged_inner_com = lariac_usc_df.overlay(parcels_usc_com_df, how='intersection')

# spatial join the 2d grid and the neighborhood boundary
grid_usc_com = gpd.sjoin(ah_grid, usage_usc_com)

# plot to visualize
ax = lariac_usc_merged_inner_com.plot(figsize=(10,10), alpha=0.5, facecolor = 'none', edgecolor='blue')
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
ah_grid.plot(ax=ax, facecolor="none", edgecolor='gray')
usage_usc_com.plot(ax=ax, alpha=0.1, edgecolor='k')

In [ ]:
# drop columns with 'index_*'
lariac_usc_merged_inner_com.drop(list(lariac_usc_merged_inner_com.filter(regex = 'index_')), axis=1, inplace=True)
grid_usc_com.drop(list(grid_usc.filter(regex = 'index_')), axis=1, inplace=True)
# add extra geometry column to save grid geometry for spatial joins
grid_usc_com['grid_geometry'] = grid_usc_com.geometry
# make the index a column
grid_usc_com.reset_index(inplace=True)
# rename index to grid_cell_index
grid_usc_com.rename(columns={'index':'grid_cell_index'}, inplace=True)

In [ ]:
#testing overlay 
buildings_usc_overlay_intersection_com = lariac_usc_merged_inner_com.overlay(grid_usc_com, how='intersection')
# plot to visualize
#buildings_usc_overlay_intersection_com.explore()

In [ ]:
# total building area in this neighborhood?
total_building_area_usc_com = buildings_usc_overlay_intersection_com['AREA'].sum()

# calculate building volume and add as a new column
buildings_usc_overlay_intersection_com['VOLUME'] = buildings_usc_overlay_intersection_com['AREA']*buildings_usc_overlay_intersection_com['HEIGHT'] # units: cubic feet

# add column that describes the fraction of the whole building that segment represents
buildings_usc_overlay_intersection_com['building_fraction'] = buildings_usc_overlay_intersection_com.to_crs('epsg:3857').geometry.area/buildings_usc_overlay_intersection_com['shape_area_whole']
buildings_usc_overlay_intersection_com['VOLUME'] = buildings_usc_overlay_intersection_com['building_fraction']*buildings_usc_overlay_intersection_com['VOLUME']

# total building area in this neighborhood?
total_building_vol_usc_com = buildings_usc_overlay_intersection_com['VOLUME'].sum()

# df of total building volume in each grid cell
building_volume_df_com = buildings_usc_overlay_intersection_com[['grid_cell_index', 'VOLUME']]
volume_by_cell_com = building_volume_df_com.groupby('grid_cell_index').sum()
# make the index a column
volume_by_cell_com.reset_index(inplace=True)

# add cells with no buildings 
volume_by_cell_all_com = volume_by_cell_com.merge(grid_usc_com, how='outer') # outer join
volume_by_cell_all_com = volume_by_cell_all_com[['grid_cell_index','VOLUME','pct_elec','usage_joules','grid_geometry']] # only keep these columns

In [ ]:
building_volume_df_com[building_volume_df_com['grid_cell_index']==687]

In [ ]:
# calculate volume-based weights and finally, the AHF
ahf_com = gpd.GeoDataFrame(volume_by_cell_all_com, geometry='grid_geometry')
ahf_com['VOLUME'] = ahf_com['VOLUME'].fillna(0)
ahf_com['vol_frac'] = ahf_com['VOLUME']/total_building_vol_usc_com
ahf_com['electricity_use'] = ahf_com['pct_elec']*ahf_com['usage_joules']*ahf_com['vol_frac']
ahf_com['ahf'] = ahf_com['electricity_use']/cell_area/seconds_in_year
ahf_com

In [ ]:
ahf_com[ahf_com['ahf']>20]

In [ ]:
ahf_com.head()

In [ ]:
# plot to visualize
ax = ahf_com.plot(figsize=(10,10), column='ahf', alpha=0.7, legend=True, cmap='YlOrRd')#, vmin=0, vmax=5, legend_kwds={'label':'Anthropogenic heat flux [W/$m^2$]'})
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
grid_usc_com.plot(ax=ax, facecolor="none", edgecolor='gray')
buildings_usc_overlay_intersection_com.plot(ax=ax, facecolor="none", edgecolor='blue', alpha=0.4)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

In [ ]:
"""
I noticed that some AHF values for some cells seemed to be suspiciously large.
Turns out this is due to mixed-use buildings and some parking lots included as commercial structures. 
Temporary solution: weight by building footprint instead of volume until more permanent solution.
"""
# === Weight by area ===
# total building area in this neighborhood?
total_building_area_usc_com = buildings_usc_overlay_intersection_com['AREA'].sum()

# add column that describes the fraction of the whole building that segment represents
buildings_usc_overlay_intersection_com['building_fraction'] = buildings_usc_overlay_intersection_com.to_crs('epsg:3857').geometry.area/buildings_usc_overlay_intersection_com['shape_area_whole']
buildings_usc_overlay_intersection_com['AREA'] = buildings_usc_overlay_intersection_com['building_fraction']*buildings_usc_overlay_intersection_com['AREA']

# df of total building volume in each grid cell
building_area_df_com = buildings_usc_overlay_intersection_com[['grid_cell_index', 'AREA']]
area_by_cell_com = building_area_df_com.groupby('grid_cell_index').sum()
# make the index a column
area_by_cell_com.reset_index(inplace=True)

# add cells with no buildings 
area_by_cell_all_com = area_by_cell_com.merge(grid_usc_com, how='outer') # outer join
area_by_cell_all_com = area_by_cell_all_com[['grid_cell_index','AREA','pct_elec','usage_joules','grid_geometry']] # only keep these columns

In [ ]:
# calculate AREA-based weights and finally, the AHF
ahf_com['AREA'] = area_by_cell_all_com['AREA']
ahf_com['AREA'] = ahf_com['AREA'].fillna(0)
ahf_com['area_frac'] = ahf_com['AREA']/total_building_area_usc_com
ahf_com['electricity_use_area_weighted'] = ahf_com['pct_elec']*ahf_com['usage_joules']*ahf_com['area_frac']
ahf_com['ahf_area_weighted'] = ahf_com['electricity_use_area_weighted']/cell_area/seconds_in_year
ahf_com.head()

In [ ]:
# plot to visualize
ax = ahf_com.plot(figsize=(10,10), column='ahf_area_weighted', alpha=0.7, legend=True, cmap='YlOrRd', vmin=0, vmax=6, legend_kwds={'label':'Anthropogenic heat flux [W/$m^2$]'})
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
grid_usc_com.plot(ax=ax, facecolor="none", edgecolor='gray')
buildings_usc_overlay_intersection_com.plot(ax=ax, facecolor="none", edgecolor='blue', alpha=0.4)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

In [ ]:
ahf.head()

In [ ]:
ahf_com.head()

In [ ]:
# plot AHF with naive averaging (i.e., no building volume weights)
ahf_com['electricity_use_naive'] = (ahf_com['pct_elec']*ahf_com['usage_joules'])/n_cells
ahf_com['ahf_naive'] = ahf_com['electricity_use_naive']/cell_area/seconds_in_year
ax = ahf_com.plot(figsize=(10,10), column='ahf_naive', alpha=0.7, legend=True, cmap='YlOrRd', vmin=0, vmax=6,legend_kwds={'label':'Anthropogenic heat flux [W/$m^2$]'})
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
grid_usc_com.plot(ax=ax, facecolor="none", edgecolor='gray')
buildings_usc_overlay_intersection_com.plot(ax=ax, facecolor="none", edgecolor='blue', alpha=0.2)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

## Commercial + Residential Combined

### Proof of concept: University Park neighborhood

In [ ]:
ahf_combined = ahf.copy()
ahf_combined['ahf'] = ahf['ahf'] + ahf_com['ahf_area_weighted']
ahf_combined['ahf_naive'] = ahf['ahf_naive'] + ahf_com['ahf_naive']

In [ ]:
# plot with building weighting applied
ax = ahf_combined.plot(figsize=(10,10), column='ahf', alpha=0.7, legend=True, cmap='YlOrRd', vmin=0, vmax=6, legend_kwds={'label':'Anthropogenic heat flux [W/$m^2$]'})
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
grid_usc.plot(ax=ax, facecolor="none", edgecolor='gray')
buildings_usc_overlay_intersection.plot(ax=ax, facecolor="none", edgecolor='blue', alpha=0.2)
buildings_usc_overlay_intersection_com.plot(ax=ax, facecolor="none", edgecolor='blue', alpha=0.2)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))

In [ ]:
# plot without building weighting (i.e., naive)
ax = ahf_com.plot(figsize=(10,10), column='ahf_naive', alpha=0.7, legend=True, cmap='YlOrRd', vmin=0, vmax=6, legend_kwds={'label':'Anthropogenic heat flux [W/$m^2$]'})
ax.get_xaxis().get_major_formatter().set_useOffset(False)
ctx.add_basemap(ax, source=ctx.providers.Stamen.Toner, crs=lariac_usc_merged_inner.crs) # notice the crs must match between basemap and data
grid_usc.plot(ax=ax, facecolor="none", edgecolor='gray')
buildings_usc_overlay_intersection.plot(ax=ax, facecolor="none", edgecolor='blue', alpha=0.2)
buildings_usc_overlay_intersection_com.plot(ax=ax, facecolor="none", edgecolor='blue', alpha=0.2)
ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))